# Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Install Library

In [ ]:
!pip install pyarrow==15.0.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 13.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 1.40.0 requires pyarrow>=15.0.2, but you have pyarrow 15.0.0 which is incompatible.


In [ ]:
!pip install pyarrow==15.0.0
!pip install datasets sentencepiece --quiet
!pip install transformers[torch] --quiet
!pip install peft --quiet
!pip install evaluate --quiet
!pip install -U bitsandbytes --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 1.40.0 requires pyarrow>=15.0.2, but you have pyarrow 15.0.0 which is incompatible.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12=

In [ ]:
import transformers
import numpy as np
import json
import os
from torch.utils.data import Dataset, random_split
import torch
from peft import get_peft_model, LoraConfig, TaskType

In [ ]:
from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
if torch.cuda.is_available():
    print("CUDA is available. Using GPU.")
    device = torch.device("cuda")
else:
    print("CUDA not available. Using CPU.")
    device = torch.device("cpu")

CUDA is available. Using GPU.


# Load Pre-train Model

In [ ]:
from transformers import T5Tokenizer, AutoModelForSeq2SeqLM

pretrain_model_name = "Peenipat/ThaiT5-Instruct"

tokenizer = T5Tokenizer.from_pretrained(pretrain_model_name)

tokenizer_config.json:   0%|          | 0.00/19.2k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/784k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/416 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.25M [00:00<?, ?B/s]

In [ ]:
SPECIAL_TOKEN = ['<SYMBOL>','<ASPECT>','<OPINION>','<POS>','<NEG>','<NEU>']
special_tokens_dict = {'additional_special_tokens': SPECIAL_TOKEN}
num_added_tokens = tokenizer.add_special_tokens(special_tokens_dict)

In [ ]:
from transformers import BitsAndBytesConfig
import torch

In [ ]:
use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False

In [ ]:
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

In [ ]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM,
)

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(pretrain_model_name)
model.resize_token_embeddings(len(tokenizer))

config.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/977M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
The new lm_head weights will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Embedding(30006, 768)

#Load Dataset

In [ ]:
dataset_folder = '/content/drive/MyDrive/KMITL/Project/Data'
dataset_filename = 'dataset_for_finetune_merged.csv'

In [ ]:
from datasets import load_dataset
dataset = load_dataset('csv', data_files=os.path.join(dataset_folder, dataset_filename))

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 4749
    })
})

In [ ]:
dataset['train'][0]

{'text': '"AAV" เข้าสู่ช่วง Turnaround โบรกฯ ชี้เด่นสุด ฟรีวีซ่าไทย-จีน',
 'label': '<SYMBOL> AAV <ASPECT> เข้าสู่ช่วง <OPINION> Turnaround <POS>\n<SYMBOL> AAV <ASPECT> โบรกฯ ชี้ <OPINION> เด่น <POS>\n<SYMBOL> AAV <ASPECT> เข้าสู่ช่วง <OPINION> Turnaround <POS>\n<SYMBOL> AAV <ASPECT> โบรกฯ ชี้ <OPINION> เด่นสุด <POS>'}

In [ ]:
def process_data_to_model_inputs(batch):
    input_encoding = tokenizer(batch["text"], padding="max_length", truncation=True, max_length=512)
    batch["input_ids"] = input_encoding.input_ids
    batch["attention_mask"] = input_encoding.attention_mask
    output_encoding = tokenizer(batch["label"], padding="max_length", truncation=True, max_length=512)
    batch["labels"] = output_encoding.input_ids
    batch["decoder_attention_mask"] = output_encoding["attention_mask"]
    return batch

In [ ]:
tokenized_dataset = dataset.map(process_data_to_model_inputs, batched=True)

Map:   0%|          | 0/4749 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask', 'labels', 'decoder_attention_mask'],
        num_rows: 4749
    })
})

In [ ]:
tokenized_dataset['train'].train_test_split(test_size=0.2)

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask', 'labels', 'decoder_attention_mask'],
        num_rows: 3799
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask', 'labels', 'decoder_attention_mask'],
        num_rows: 950
    })
})

In [ ]:
tokenized_dataset = tokenized_dataset.remove_columns(['text','label'])

In [ ]:
datacollator = lambda data: {
                              'input_ids': data['input_ids'],
                              'labels': data['labels'],
                              }

In [ ]:
def split_dataset(dataset, train_size=0.8, test_size=0.1, seed=42):

    val_size = 1 - (train_size + test_size)

    if train_size + test_size + val_size != 1:
        raise ValueError("Train, test, and validation sizes must sum up to 1.")

    dataset = dataset["train"].train_test_split(test_size=test_size + val_size, seed=seed)

    test_val_dataset = dataset['test'].train_test_split(test_size=val_size / (test_size + val_size), seed=seed)

    return {
        "train": dataset["train"],
        "validation": test_val_dataset["train"],
        "test": test_val_dataset["test"]
    }

In [ ]:
splits = split_dataset(tokenized_dataset)

In [ ]:
splits

{'train': Dataset({
     features: ['input_ids', 'attention_mask', 'labels', 'decoder_attention_mask'],
     num_rows: 3799
 }),
 'validation': Dataset({
     features: ['input_ids', 'attention_mask', 'labels', 'decoder_attention_mask'],
     num_rows: 475
 }),
 'test': Dataset({
     features: ['input_ids', 'attention_mask', 'labels', 'decoder_attention_mask'],
     num_rows: 475
 })}

#Fine-tuning

In [ ]:
project_root = '/content/drive/MyDrive/KMITL/Project/Model'
model_folder = 'thaiT5_absa_model_v2'
model_path = os.path.join(project_root, model_folder)
os.makedirs(model_path, exist_ok=True)
model_path

'/content/drive/MyDrive/KMITL/Project/Model/thaiT5_absa_model_v2'

In [ ]:
from transformers import Trainer, TrainingArguments

In [ ]:
training_args = TrainingArguments(output_dir=os.path.join(model_path,'model_output'),
                                  num_train_epochs=10,
                                  logging_steps=100,
                                  per_device_train_batch_size=8,
                                  per_device_eval_batch_size=8,
                                  warmup_steps=500,
                                  weight_decay=0.05,
                                  logging_dir=os.path.join(model_path,'logs'),
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=splits["train"],
    eval_dataset=splits["validation"],
)

In [ ]:
for param in model.parameters():
    param.data = param.data.contiguous()

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: 64070271 (64070271-king-mongkut-s-institute-of-technology-ladkrabang) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
100,7.147400
200,0.232700
300,0.200200
400,0.164800
500,0.149700
600,0.129600
700,0.084800
800,0.057600
900,0.050300
1000,0.038100


TrainOutput(global_step=4750, training_loss=0.18554746182341325, metrics={'train_runtime': 2758.0987, 'train_samples_per_second': 13.774, 'train_steps_per_second': 1.722, 'total_flos': 2.582373769150464e+16, 'train_loss': 0.18554746182341325, 'epoch': 10.0})

# Save Model

In [ ]:
saved_model_path = os.path.join(model_path, 'save_model')
model.save_pretrained(saved_model_path)
tokenizer.save_pretrained(saved_model_path)

('/content/drive/MyDrive/KMITL/Project/Model/thaiT5_absa_model_v2/save_model/tokenizer_config.json',
 '/content/drive/MyDrive/KMITL/Project/Model/thaiT5_absa_model_v2/save_model/special_tokens_map.json',
 '/content/drive/MyDrive/KMITL/Project/Model/thaiT5_absa_model_v2/save_model/spiece.model',
 '/content/drive/MyDrive/KMITL/Project/Model/thaiT5_absa_model_v2/save_model/added_tokens.json')

In [ ]:
dataset['train'][0]

{'text': '"AAV" เข้าสู่ช่วง Turnaround โบรกฯ ชี้เด่นสุด ฟรีวีซ่าไทย-จีน',
 'label': '<SYMBOL> AAV <ASPECT> เข้าสู่ช่วง <OPINION> Turnaround <POS>\n<SYMBOL> AAV <ASPECT> โบรกฯ ชี้ <OPINION> เด่น <POS>\n<SYMBOL> AAV <ASPECT> เข้าสู่ช่วง <OPINION> Turnaround <POS>\n<SYMBOL> AAV <ASPECT> โบรกฯ ชี้ <OPINION> เด่นสุด <POS>'}

In [ ]:
def generate_text(input_text):
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)
    model.to(device)

    outputs = model.generate(
        input_ids,
        num_beams=4,
        do_sample=False,
        max_length=256,
        eos_token_id=tokenizer.eos_token_id
    )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=False)

    start_idx = decoded.find("<SYMBOL>")
    end_idx = decoded.find("</s>", start_idx)
    result = decoded[start_idx:end_idx].strip() if start_idx != -1 else decoded.strip()

    return result

In [ ]:
input_text = "AOT ราคาดีดขึ้น โบรกฯ คาดกำไร Q1 โต 1,577% แนวโน้มยังดีต่อเนื่อง"
print(generate_text(input_text))

<SYMBOL> AOT <ASPECT> ราคา <OPINION> ดีดขึ้น <POS> <SYMBOL> AOT <ASPECT> กําไร <OPINION> โต <POS> <SYMBOL> AOT <ASPECT> แนวโน้ม <OPINION> ดีต่อเนื่อง <POS>


In [ ]:
tokenizer.decode(tokenizer('<SYMBOL> PTTGC <ASPECT> หุ้น <OPINION> ราคาร่วง <NEG>').input_ids)

'<SYMBOL> PTTGC <ASPECT> หุ้น <OPINION> ราคาร่วง <NEG> </s>'

In [ ]:
text = "<SYMBOL> PTTGC <ASPECT> หุ้น <OPINION> ราคาร่วง <NEG>"
ids = tokenizer.encode(text)
print(tokenizer.decode(ids))  # ✅ ต้องใกล้เคียงข้อความเดิม

<SYMBOL> PTTGC <ASPECT> หุ้น <OPINION> ราคาร่วง <NEG> </s>
